In [8]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from osgeo import gdal
import os
import time
import multiprocessing

In [2]:
class Dataset:
    def __init__(self, in_file):
        self.in_file = in_file  # Tiff或者ENVI文件

        dataset = gdal.Open(self.in_file)
        self.XSize = dataset.RasterXSize  # 网格的X轴像素数量
        self.YSize = dataset.RasterYSize  # 网格的Y轴像素数量
        self.Bands = dataset.RasterCount  # 波段数
        self.GeoTransform = dataset.GetGeoTransform()  # 投影转换信息
        self.ProjectionInfo = dataset.GetProjection()  # 投影信息
    
    def get_data(self):
        #band: 读取第几个通道的数据
        dataset = gdal.Open(self.in_file)
        data = dataset.ReadAsArray(0,0,self.XSize,self.YSize)
        return data
    

    def get_lon_lat(self):
        #获取经纬度信息
        gtf = self.GeoTransform
        x_range = range(0, self.XSize)
        y_range = range(0, self.YSize)
        x, y = np.meshgrid(x_range, y_range)
        lon = gtf[0] + x * gtf[1] + y * gtf[2]
        lat = gtf[3] + x * gtf[4] + y * gtf[5]
        
        lon_lat=[]
        for (longitude,latitude) in zip(lon,lat):
            lon_lat.append(list(zip(longitude,latitude)))
            
        return np.array(lon_lat)
    
    def new_dataset(self,data,lon_lat):
        new_dataset=[]
        for i in range(self.YSize):
            for j in range(self.XSize):
                x1 = lon_lat[i,j]
                x2 = data[:,i,j]
                x=np.hstack((x1,x2))
                new_dataset.append(x)
            
        return np.array(new_dataset)
    
    def dataset2dim(self,data):
        dataset2dim=[]
        for i in range(self.YSize):
            for j in range(self.XSize):
#                 x1 = lon_lat[i,j]
                x2 = data[:,i,j]
#                 x=np.hstack((x1,x2))
                dataset2dim.append(x2)

        return np.array(dataset2dim)

In [17]:
def func(x, m1, m2, m3, m4, m5, m6):
    return m1 + m2 /(1 + np.exp(-m3 * (x-m4))) - m2/(1 + np.exp(-m5 * (x-m6)))

#注意初值
def DofittingandValidation(yData, xData):
    Parameters, pcov = curve_fit(func, xData, yData, p0=[900,5000,0.1,140,0.1,270], maxfev=100000000)#,method='trf', maxfev=1000000)
    return Parameters

In [5]:
start=time.time()

dir_path = r"D:\Desktop\mypaper\data"
filename = "gee-LAI-10884.tif"
file_path = os.path.join(dir_path, filename)
dataset = Dataset(file_path)

data = dataset.get_data( ) 

lon_lat = dataset.get_lon_lat()  # 获取经纬度信息longitude, latitude
# new_dataset=dataset.new_dataset(data,lon_lat)
dataset2dim=dataset.dataset2dim(data)

In [6]:
print(dataset2dim.shape)

(12656, 92)


In [18]:

xdim=dataset2dim.shape[0]
ydim=dataset2dim.shape[1]

print(xdim,ydim)

pInput=[900,5000,0.1,140,0.1,270]
pData=np.repeat([pInput],xdim,axis=0)
print('pData',pData.shape)
xInput=np.linspace(1, 365, 92)
xData=np.repeat([xInput],xdim,axis=0)
print('xData',xData.shape)


12656 92
pData (12656, 6)
xData (12656, 92)


In [ ]:
result=np.array(list(map(DofittingandValidation,dataset2dim,xData)))
print('total time',time.time()-start)
print(result.shape)
print(result)

d:\python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
d:\python37\lib\site-packages\scipy\optimize\minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
